In [ ]:
import glob

files = glob.glob('img/*.jpg')

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([img_to_array(load_img(f)) for f in files])

imgs.shape

In [ ]:
import os

th = 128

labels = np.array([img_to_array(load_img(f"{os.path.splitext(f)[0]}.gif", color_mode = 'grayscale')) for f in files])

labels[labels < th] = 0
labels[labels >= th] = 1

labels.shape


In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, UpSampling2D, concatenate
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPool2D
from keras.layers.normalization import BatchNormalization

input = Input(shape = imgs.shape[1:])

x = input

x = BatchNormalization()(x)

x = Conv2D(16, 3, padding='same', activation = 'relu')(x)
x = Conv2D(16, 3, padding='same', activation = 'relu')(x)
x = MaxPool2D()(x)

x1 = Conv2D(8, 1, activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(32, 3, padding='same', activation = 'relu')(x)
x = Conv2D(32, 3, padding='same', activation = 'relu')(x)
x = MaxPool2D()(x)

x2 = Conv2D(16, 1, activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(64, 3, padding='same', activation = 'relu')(x)
x = Conv2D(64, 3, padding='same', activation = 'relu')(x)
x = MaxPool2D()(x)

x3 = Conv2D(32, 1, activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(128, 3, padding='same', activation = 'relu')(x)
x = Conv2D(128, 3, padding='same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(256, 3, padding='same', activation = 'relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(128, 3, padding='same', activation = 'relu')(x)
x = Conv2DTranspose(128, 3, padding='same', activation = 'relu')(x)

x = concatenate([x, x3])

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(64, 3, padding='same', activation = 'relu')(x)
x = Conv2DTranspose(64, 3, padding='same', activation = 'relu')(x)

x = concatenate([x, x2])

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(32, 3, padding='same', activation = 'relu')(x)
x = Conv2DTranspose(32, 3, padding='same', activation = 'relu')(x)

x = concatenate([x, x1])

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(16, 3, padding='same', activation = 'relu')(x)
x = Conv2DTranspose(16, 3, padding='same', activation = 'relu')(x)

x = Dropout(0.3)(x)

x = Dense(16, activation = 'relu')(x)
x = Dense(8, activation = 'relu')(x)

output = Dense(labels.shape[-1], activation = 'sigmoid')(x)

model = Model(inputs = input, outputs = output)

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

In [ ]:
hist = model.fit(imgs, labels, initial_epoch = 0, epochs = 100, batch_size = 10)

hist

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (8, 4)

plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'])

plt.subplot(1, 2, 2)
plt.plot(hist.history['acc'])

In [ ]:
def predict(index, s = 14.0):
    plt.rcParams['figure.figsize'] = (s, s)

    pred = model.predict(np.array([imgs[index]]))[0]
    
    plt.subplot(1, 4, 1)
    plt.imshow(imgs[index].astype(int))

    size = tuple(labels.shape[1:-1])
    
    plt.subplot(1, 4, 2)
    plt.imshow(labels[index].reshape(size), cmap = 'gray')

    plt.subplot(1, 4, 3)
    plt.imshow(pred.reshape(size), cmap = 'gray')

    predbin = pred.reshape(size)
    predbin[predbin < 0.5] = 0
    predbin[predbin >= 0.5] = 1

    plt.subplot(1, 4, 4)
    plt.imshow(predbin, cmap = 'gray')

In [ ]:
model.save('model/cnn_encoder-decoder2_1_0.h5')

In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)

In [ ]:
predict(3)